In [1]:
!pip install transformers[torch]
!pip install accelerate -U
!pip install sentencepiece

In [2]:
import warnings
warnings.filterwarnings("ignore", category=UserWarning, module='transformers.generation.utils')

In [3]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [4]:
path='/content/drive/MyDrive/GYAFC_Corpus'

In [5]:
import pandas as pd

In [39]:
df_test=pd.DataFrame()

In [74]:
df_test['Casual']=pd.read_csv('drive/MyDrive/GYAFC_Corpus/GYAFC_Corpus/Entertainment_Music/test/informal.ref0',sep='\t')

In [75]:
df_test['Formal']=pd.read_csv('/content/drive/MyDrive/GYAFC_Corpus/GYAFC_Corpus/Entertainment_Music/test/formal',sep='\t')

In [42]:
df_train_Entertainment_Music=pd.DataFrame()

In [43]:
df_train_Family_Relationships=pd.DataFrame()

In [44]:
df_train_Entertainment_Music['Casual']=pd.read_csv('drive/MyDrive/GYAFC_Corpus/GYAFC_Corpus/Entertainment_Music/train/informal',sep='\t')

In [45]:
df_train_Entertainment_Music['Formal']=pd.read_csv('drive/MyDrive/GYAFC_Corpus/GYAFC_Corpus/Entertainment_Music/train/formal',sep='\t')

In [46]:
df_train_Family_Relationships['Casual']=pd.read_csv('drive/MyDrive/GYAFC_Corpus/GYAFC_Corpus/Family_Relationships/train/informal',sep='\t')

In [47]:
df_train_Family_Relationships['Formal']=pd.read_csv('drive/MyDrive/GYAFC_Corpus/GYAFC_Corpus/Family_Relationships/train/formal',sep='\t')

In [48]:
df_train= pd.concat([df_train_Entertainment_Music,df_train_Family_Relationships], ignore_index=True)

In [49]:
df_train.head(-1)

,Casual,Formal
0,that page did not give me viroses(i think),I don't think that page gave me viruses.
1,"of corse i be wachin it evry day, my fav chara...","I watch it everyday, my favorite charachter is..."
2,runescape.com (my kids love it) & funbrain.com...,Funbrain.com and runescape.com are great for f...
3,Is he gay?He was on Late Night with Conan O'Br...,He was on the Late Night show with Conan O'Bri...
4,"id have to say mel gibson, a strong [god] beli...",Mel Gibson is a strong believed of God.
...,...,...
104554,is he alot older than you?,Is he much older than you are?
104555,of corse it depends on what relation u r looki...,"Of course, it depends on what type of relation..."
104556,Wear a sign that says Hi!,"Wear a sign that say ""Hi!"""
104557,"I don't believe in playing games, I hate when ...",I do not like when guys play games with me.


In [50]:
df_train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 104560 entries, 0 to 104559
Data columns (total 2 columns):
 #   Column  Non-Null Count   Dtype 
---  ------  --------------   ----- 
 0   Casual  104560 non-null  object
 1   Formal  104558 non-null  object
dtypes: object(2)
memory usage: 1.6+ MB


In [54]:
df_train = df_train.dropna(subset=['Casual', 'Formal'])

In [55]:
df_test.head()

,Casual,Formal
0,"aspen colorado has he best music festivals, yo...",There's nothing he needs to change.
1,You can get almost anything on ebay!,It does not exist.
2,everybody is Dying to get in,Mine is book by Steve Martin called 'The Pleas...
3,not idiots like 50 cent and his whole Gay unit...,What differentiates a mosquitoo from a blonde?
4,different from what I've seen though,"They're pretty good. Also, that's a good song."


In [56]:
df_test.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1415 entries, 0 to 1414
Data columns (total 2 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   Casual  1415 non-null   object
 1   Formal  1081 non-null   object
dtypes: object(2)
memory usage: 22.2+ KB


In [57]:
import torch
from transformers import T5ForConditionalGeneration, T5Tokenizer, TextDataset, DataCollatorForLanguageModeling, Trainer, TrainingArguments

In [22]:
df_train['combined'] = "[Casual] " + df_train['informal'] + "[Formal] " + df_train['formal']
df_train['combined'].to_csv('training_data.txt', index=False, header=False)

In [23]:
tokenizer = T5Tokenizer.from_pretrained('t5-small')
model_t5 = T5ForConditionalGeneration.from_pretrained('prithivida/informal_to_formal_styletransfer')

tokenizer_config.json:   0%|          | 0.00/2.32k [00:00<?, ?B/s]

spiece.model:   0%|          | 0.00/792k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.39M [00:00<?, ?B/s]

You are using the default legacy behaviour of the <class 'transformers.models.t5.tokenization_t5.T5Tokenizer'>. This is expected, and simply means that the `legacy` (previous) behavior will be used so nothing changes for you. If you want to use the new behaviour, set `legacy=False`. This should only be set if you understand what it means, and thouroughly read the reason why this was added as explained in https://github.com/huggingface/transformers/pull/24565
Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


config.json:   0%|          | 0.00/1.39k [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/892M [00:00<?, ?B/s]

In [24]:
train_dataset = TextDataset(
    tokenizer=tokenizer,
    file_path='training_data.txt',
    block_size=512
)

/usr/local/lib/python3.10/dist-packages/transformers/data/datasets/language_modeling.py:53: FutureWarning: This dataset will be removed from the library soon, preprocessing should be handled with the 🤗 Datasets library. You can have a look at this example script for pointers: https://github.com/huggingface/transformers/blob/main/examples/pytorch/language-modeling/run_mlm.py
  warnings.warn(


In [25]:
data_collator = DataCollatorForLanguageModeling(
    tokenizer=tokenizer,
    mlm=False
)

In [26]:
training_args = TrainingArguments(
    output_dir='./t5-formal',
    overwrite_output_dir=True,
    num_train_epochs=10,
    per_device_train_batch_size=4,
    save_steps=10_000,
    save_total_limit=2,
)

In [27]:
trainer = Trainer(
    model=model_t5,
    args=training_args,
    data_collator=data_collator,
    train_dataset=train_dataset,
)

In [29]:
trainer.train()

Step,Training Loss
500,0.415800
1000,0.014900
1500,0.008900
2000,0.006700
2500,0.005100
3000,0.003900
3500,0.003400
4000,0.003300
4500,0.003000
5000,0.003100


TrainOutput(global_step=18180, training_loss=0.013738441710310145, metrics={'train_runtime': 6809.7142, 'train_samples_per_second': 10.676, 'train_steps_per_second': 2.67, 'total_flos': 4.4271238643712e+16, 'train_loss': 0.013738441710310145, 'epoch': 10.0})

In [33]:
model_t5.save_pretrained('./t5-formal')
tokenizer.save_pretrained('./t5-formal')

('./t5-formal/tokenizer_config.json',
 './t5-formal/special_tokens_map.json',
 './t5-formal/spiece.model',
 './t5-formal/added_tokens.json')

In [34]:
tokenizer = T5Tokenizer.from_pretrained('./t5-formal')
model = T5ForConditionalGeneration.from_pretrained('./t5-formal')

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


In [37]:
input_sentence = "informal: that page did not give me viroses(i think)"
input_ids = tokenizer.encode(input_sentence, return_tensors='pt')
output = model.generate(input_ids)
print(tokenizer.decode(output[0], skip_special_tokens=False))

<pad>informal: that page did not give me viroses(i think) that page did not


In [38]:
df_train.head()

,informal,formal,combined
0,that page did not give me viroses(i think),I don't think that page gave me viruses.,informal: that page did not give me viroses(i ...
1,"of corse i be wachin it evry day, my fav chara...","I watch it everyday, my favorite charachter is...","informal: of corse i be wachin it evry day, my..."
2,runescape.com (my kids love it) & funbrain.com...,Funbrain.com and runescape.com are great for f...,informal: runescape.com (my kids love it) & fu...
3,Is he gay?He was on Late Night with Conan O'Br...,He was on the Late Night show with Conan O'Bri...,informal: Is he gay?He was on Late Night with ...
4,"id have to say mel gibson, a strong [god] beli...",Mel Gibson is a strong believed of God.,"informal: id have to say mel gibson, a strong ..."


In [60]:
input_sentence = "informal: it gives us something to do!"
input_ids = tokenizer.encode(input_sentence, return_tensors='pt')

# Set generation parameters
output = model.generate(
    input_ids,
    max_length=50,  # Adjust as needed
    num_beams=5,    # Using beam search for better quality
    early_stopping=True
)

# Decode and print the output
output_text = tokenizer.decode(output[0], skip_special_tokens=True)
# Strip off the input part from the output
output_text = output_text.replace(input_sentence, '').strip()
print(output_text)


In [61]:
print(output_text)

In [64]:
input_sentence = "informal: that page didnt give me viroses"
input_ids = tokenizer.encode(input_sentence, return_tensors='pt')

output = model.generate(
    input_ids,
    max_length=50,
    num_beams=5,
    early_stopping=True,
    no_repeat_ngram_size=2  # Prevents the model from repeating n-grams
)

output_text = tokenizer.decode(output[0], skip_special_tokens=True)

# Post-process to remove echoed input
start_token = "formal: "
if start_token in output_text:
    output_text = output_text.split(start_token)[-1]

print(output_text.strip())


that page didn't give me viroses.


In [76]:
df_test.head()

,Casual,Formal
0,He dosn't need to change one thing!,There's nothing he needs to change.
1,Because there is no such thing!!!!!!!!!,It does not exist.
2,mine is the pleasure of my company by steve ma...,Mine is book by Steve Martin called 'The Pleas...
3,whats the difference between a mosquito and a ...,What differentiates a mosquitoo from a blonde?
4,yep they are all right.. good song hey.,"They're pretty good. Also, that's a good song."


In [78]:
import torch
from transformers import T5ForConditionalGeneration, T5Tokenizer

In [79]:
model_t5 = T5ForConditionalGeneration.from_pretrained('./t5-formal')
tokenizer_t5 = T5Tokenizer.from_pretrained('./t5-formal')

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


In [80]:
def convert_informal_to_formal_t5(informal_sentence):
    input_text = f"convert informal to formal: {informal_sentence}"
    input_ids = tokenizer_t5.encode(input_text, return_tensors='pt')

    # Generate the output (you may adjust max_length and other parameters as needed)
    output_ids = model_t5.generate(input_ids, max_length=50, num_beams=5, early_stopping=True)

    # Decode and return the formal sentence
    formal_sentence = tokenizer_t5.decode(output_ids[0], skip_special_tokens=True)
    return formal_sentence

In [83]:
df_test_mini=df_test.head(100)

In [84]:
df_test_mini['Formal_T5'] = df_test_mini['Casual'].apply(convert_informal_to_formal_t5)

<ipython-input-84-53646d75ae33>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_test_mini['Formal_T5'] = df_test_mini['Casual'].apply(convert_informal_to_formal_t5)


In [85]:
!pip install sacrebleu pyter3

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 106.4/106.4 kB 3.4 MB/s eta 0:00:00


In [88]:
import sacrebleu
hypotheses = df_test_mini['Formal_T5'].tolist()
references = [df_test_mini['Formal'].tolist()]

bleu_score = sacrebleu.corpus_bleu(hypotheses, references).score
print(f"BLEU Score: {bleu_score}")

BLEU Score: 12.200214859494052


In [90]:
import pyter
def ter_score(hypotheses, references):
    total_ter = 0
    for hyp, ref in zip(hypotheses, references):
        total_ter += pyter.ter(hyp.split(), ref.split())
    return total_ter / len(hypotheses)

# Calculate TER score
ter = ter_score(df_test_mini['Formal_T5'], df_test_mini['Formal'])
print(f"TERp Score: {ter}")

TERp Score: 1.1729619727873866


In [91]:
import pyter
from transformers import T5ForConditionalGeneration, T5Tokenizer

In [92]:
tokenizer = T5Tokenizer.from_pretrained('prithivida/informal_to_formal_styletransfer')
model = T5ForConditionalGeneration.from_pretrained('prithivida/informal_to_formal_styletransfer')


tokenizer_config.json:   0%|          | 0.00/1.89k [00:00<?, ?B/s]

spiece.model:   0%|          | 0.00/792k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/1.79k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.39M [00:00<?, ?B/s]

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


In [93]:
def convert_informal_to_formal_t5(informal_sentence):
    input_text = f"paraphrase: {informal_sentence} </s>"
    input_ids = tokenizer.encode(input_text, return_tensors='pt')

    output_ids = model.generate(input_ids, max_length=128, num_beams=5, early_stopping=True)
    formal_sentence = tokenizer.decode(output_ids[0], skip_special_tokens=True)
    return formal_sentence

In [95]:
df_test_mini['Converted'] = df_test_mini['Casual'].apply(convert_informal_to_formal_t5)

/usr/local/lib/python3.10/dist-packages/transformers/models/t5/tokenization_t5.py:303: UserWarning: This sequence already has </s>. In future versions this behavior may lead to duplicated eos tokens being added.
  warnings.warn(
<ipython-input-95-3f96a4c817a7>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_test_mini['Converted'] = df_test_mini['Casual'].apply(convert_informal_to_formal_t5)


In [99]:
hypotheses = df_test_mini['Converted'].tolist()
references = [df_test_mini['Formal'].tolist()]

bleu_score = sacrebleu.corpus_bleu(hypotheses, references).score

In [100]:
ter_scores = [pyter.ter(hyp.split(), ref.split()) for hyp, ref in zip(df_test_mini['Converted'], df_test_mini['Formal'])]
ter_score = sum(ter_scores) / len(ter_scores)

In [101]:
print(f"BLEU Score: {bleu_score}")
print(f"TER Score: {ter_score}")

BLEU Score: 28.511237541142666
TER Score: 0.6966883204290659
